<span style="font-size: 3rem; font-weight: bold;">SMD Hands-On Schätzen</span>

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib widget

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
plt.rcParams['figure.figsize'] = (7.5, 5)
plt.rcParams['figure.constrained_layout.use'] = True

In [ ]:
def cov_to_corr(cov):
    '''Convert covariance to correlation matrix
    
    Taken from: https://math.stackexchange.com/a/300775/892886
    '''
    D = np.diag(1 / np.sqrt(np.diag(cov)))
    return D @ cov @ D

# Methode der kleinsten Quadrate

## Analytische Lösung für Linearkombinationen von Funktionen

(Analog zu Aufgabe 27 des letzten Übungszettels)

Hier werden wir eine Funktion der Form

$$
f(x) = p_0 + p_1 \cdot \sin(x) + p_2 \cdot \cos(x)
$$

an Daten anpassen.

Diese Funktion ist eine Linearkombination von Basisfunktionen:
$$
f(x) = \sum_{i=0}^2 p_i f_i(x)
$$

mit 

$$
f_0(x) = 1, \quad f_1(x) = \sin(x), \quad f_2(x) = \cos(x)
$$

Für diesen Fall gilt die in der Vorlesung und den Übungen vorgestellte analytische Lösung für das Problem der Kleinsten Quadrate.

In [ ]:
def linear_combination(x, funcs, parameters):
    '''Evaluate a liner combination of base functions
    
    Parameters
    ----------
    x: number or np.ndarray
        The point or points at which to evaluate
    funcs: iterable of callables
        The base functions
    parameters: iterable of numbers
        The coefficients
    '''
    if len(funcs) != len(parameters):
        raise ValueError('len(funcs) must be equal to len(parameters)')
        
    return np.sum([p * f(x) for p, f in zip(parameters, funcs)], axis=0)


funcs = [np.ones_like, np.sin, np.cos]

# create some randomized example data points
rng = np.random.default_rng(1337)
N = 100
true_parameters = np.array([2, 1, 0.5])
y_unc = rng.uniform(0.1, 0.4, N)

x = np.linspace(0, 4 * np.pi, N)
y = linear_combination(x, funcs, true_parameters)

# Messunsicherheit simulieren
y += rng.normal(0, y_unc)

plt.figure()
plt.errorbar(x, y, yerr=y_unc, ls='', marker='.')
plt.xlabel('x')
plt.ylabel('y')

Aufstellen der Design Matrix

In [ ]:
A = np.column_stack([f(x) for f in funcs])
A[:5]

Aufstellen der Gewichtsmatrix $\boldsymbol{W} = \mathrm{Cov}^{-1}(\mathbf{y})$ der Messwerte.

Wir gehen hier davon aus, dass wir die Unsicherheit jedes Messpunktes kennen und das keine Korrelation zwischen den Punkten existiert. Dies ist eine starke Vereinfachung.

Die Methode der kleinsten Quadrate erzeugt verzerrte Ergebnisse, falls die Unsicherheiten aus den Messpunkten selbst geschätzt werden. Mehr dazu weiter unten in dem Myon-Beispiel.

In [ ]:
# All measurements have a known uncertainty and no correlations
W = np.diag(np.full(N, 1 / y_unc**2)) 
plt.matshow(W)
plt.colorbar()

Berechnung der Lösung und der Kovarianz der Parameter

In [ ]:
cov = np.linalg.inv(A.T @ W @ A)

parameters = cov @ A.T @ W @ y
parameters, cov

Chi-Quadrad-Über-Anzahl-Freiheitsgrade

In [ ]:
residuals = (y - A @ true_parameters)
sum_residuals = (residuals.T @ W @ residuals)

ndf = len(y) - len(true_parameters)
chisquare_ndf = sum_residuals / ndf
chisquare_ndf

Ergebnis

In [ ]:
x_fit = np.linspace(0, 4 * np.pi, 1000)
y_fit = linear_combination(x_fit, funcs, parameters)
y_truth = linear_combination(x_fit, funcs, true_parameters)
 

plt.figure()
plt.title(
    rf'$f(x) = {parameters[0]:.2f} + {parameters[1]:.2f} \cdot \sin(x) + {parameters[2]:.2f} \cdot \cos(x)'
    rf', \quad \chi^2_\mathrm{{ndf}} = {chisquare_ndf:.2f}$'
)
plt.errorbar(x, y, yerr=y_unc, ls='', marker='.', label='Daten')
plt.plot(x_fit, y_fit, label='Fit-Ergebnis')
plt.plot(x_fit, y_truth, label='Wahrheit')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()

In [ ]:
corr = cov_to_corr(cov)

fig = plt.figure(constrained_layout=True)
ax = fig.add_subplot(1, 1, 1)
m = ax.matshow(corr, cmap='RdBu_r', vmin=-1, vmax=1)
fig.colorbar(m)

## Numerische Lösung für allgemeine Funktionen


Ist die anzupassende Funktion *keine* Linearkombination einzelner Funktionen, kann die Lösung
nur numerisch gefunden werden.

Aus dem Praktikum bekannt ist die Funktion `scipy.optimize.curve_fit`, die genau dies tut.

Der verwendete Nelder-Mead-Algorithmus hat die Eigenschaft, garantiert die korrekte analytische Lösung zu finden, falls sie existiert.

- https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

### Anwendung auf lineares Beispiel

In [ ]:
from scipy.optimize import curve_fit


def func(x, p1, p2, p3):
    return p1 + p2 * np.sin(x) + p3 * np.cos(x)


# absolute_sigma prevents scaling of errors to match χ²/ndf=1
parameters_numeric, cov_numeric = curve_fit(
    func,x, y,
    sigma=np.full(N, y_unc),
    absolute_sigma=True,
)

print(parameters, parameters_numeric, cov, cov_numeric, sep='\n')

In [ ]:
x_fit = np.linspace(0, 4 * np.pi, 1000)
y_fit = linear_combination(x_fit, funcs, parameters)
y_num = linear_combination(x_fit, funcs, parameters_numeric)
y_truth = linear_combination(x_fit, funcs, true_parameters)
 

plt.figure()
plt.errorbar(x, y, yerr=y_unc, ls='', marker='.', label='Daten')
plt.plot(x_fit, y_fit, label='Fit-Ergebnis Analytisch')
plt.plot(x_fit, y_num, label='Fit-Ergebnis Numerisch', linestyle=':')
plt.plot(x_fit, y_truth, label='Wahrheit')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc='upper right')
plt.xlim(0, 15)

### Warum dann nicht einfach immer die numerische Variante nehmen?

In [ ]:
%%timeit
A = np.column_stack([f(x) for f in funcs])

ATW = A.T @ W
cov = np.linalg.inv(ATW @ A)
parameters = cov @ ATW @ y

In [ ]:
%%timeit
parameters_numeric, cov_numeric = curve_fit(func, x, y)

### Beispiel mit allgemeiner Funktion 

Aus dem Praktikum, Spaltfunktion

In [ ]:
df = pd.read_csv('resources/spalt.csv')


LASER_WAVELENGTH_NM = 632.8e-9

def theory(phi, A0, b):
    return (A0 * b * np.sinc(b * np.sin(phi) / LASER_WAVELENGTH_NM))**2


# first try with default initial guess (1 for every parameter)
p0 = None

# now with an "educated guess" based on the data and knowledge of the
# order of magnitude of the slit size
p0 = [np.sqrt(df['I'].max()) / 1e-4, 1e-4]

params, cov = curve_fit(theory, df['phi'], df['I'], p0=p0)


plt.figure()
x = np.linspace(-0.03, 0.03, 501)
plt.plot(x, theory(x, *params), label='Fit')

plt.plot(df['phi'], df['I'], '.', label='Daten')

plt.xlabel(r'$\varphi \,\, / \,\, \mathrm{rad}$')
plt.ylabel(r'$I \,\, / \,\, \mathrm{A}$')
plt.legend(loc='best')

None # to not have mpl objects in the output

# Maximimum-Likelihood-Methode

## Ungebinnter Fit von Wahrscheinlichkeitsdichten


Stark vereinfachtes Beispiel einer CERN-Analyse.

Wir suchen den Massenpeak eines Teilchens (normal-verteilt) und haben einen exponential-verteilten Untergrund.

Wir erzeugen wieder einen "Spielzeug"-Datensatz mit Monte-Carlo Methoden:

In [ ]:
rng = np.random.default_rng(42)

E_MIN = 75
E_MAX = 175

# normally distributed signal
higgs_signal = np.random.normal(126, 5, 500)

# exponentially distributed background
background = np.random.exponential(50, size=20000)

# combine signal and background
measured = np.append(higgs_signal, background)

# remove events outside of "detector range"
in_range = (E_MIN <= measured) & (measured <= E_MAX)
measured = measured[in_range]


plt.figure()
plt.hist(measured, bins=100)
plt.xlabel('$m \,/\, \mathrm{GeV}$')
plt.margins(x=0)
None

### Definition der negativen Log-Likelihood

Wir bilden eine Überlagerung von zwei Wahrscheinlichkeitsdichten, die einen Anteil von $p$ bzw. $1-p$ haben.

Wir müssen die Wahrscheinlichkeitsdichten auf das gemessene Intervall normieren.
In diesem Fall ignorieren wir dies für die Normalverteilung unter der Annahme, dass der Massenpeak vollständig im Intervall enthalten ist.

Also:

\begin{align}
P_1 &= N(\mu, \sigma) \\[1ex]
P_2 &= \frac{1}{\exp(-E_\mathrm{min} / \tau) - \exp(-E_\mathrm{max} / \tau)} \exp(- E / \tau) \\[1ex]
P(E | p, \mu, \sigma, \tau) &= p \cdot P_1(E, \mu, \sigma) + (1 - p) P_2(E | \tau)) \\[1ex]
\mathcal{L}(p, \mu, \sigma, \tau) &= \prod_i P(E_i | p, \mu, \sigma, \tau) \\[1ex]
-\log\mathcal{L}(p, \mu, \sigma, \tau) &= -\sum_i \log(P(E_i | p, \mu, \sigma, \tau))
\end{align}

Im Code, unter Verwendung der Verteilungs-Klassen aus `scipy.stats`, sieht die PDF wie folgt aus:

In [ ]:
from scipy.stats import norm, expon


def pdf(x, mean, std, tau, p):
    expon_norm = np.exp(-E_MIN / tau) - np.exp(-E_MAX / tau)
    return (
        p * norm.pdf(x, mean, std) 
        + (1 - p) / expon_norm * expon.pdf(x, scale=tau)
    )


### Lösung mit `scipy.optimize.minimize` und `numdifftools.Hessian`


Mit `scipy.optimize.minimize` können allgemeine Funktionen minimiert werden.

Diese müssen als erstes Argument ein array der zu variierenden Parameter haben.
Weitere Parameter können über das `args` Argument von `minimize` übergeben werden.

Naiv sieht dann unsere negative Log-Likelihood für die Minimierung mit scipy so aus:

In [ ]:
def neg_log_likelihood(parameters, data):
    return -np.sum(np.log(pdf(data, *parameters)))

Naiv, weil häufig analytische Umformungen der der Log-Likelihood möglich sind,  
die zu weniger Operationen führen und somit sowohl Rechenzeit sparen als auch Rundungsfehler vermeiden.

Es macht also in vielen Fällen Sinn, die Log-Likelihood per Hand aufzuschreiben und so weit es geht zu vereinfachen. Zum Beispiel auch durch das Weglassen von Termen die unter Variation der Parameter konstant sind.

Wir finden ein Minimum (und hoffentlich das globale) mit `scipy.optimize.minimize`. 

Wie aus der Vorlesung bekannt, bekommen wir einen Schätzer für die Covarianz unserer Parameter aus der Inversen der Hesse-Matrix der Likelihood, ausgewertet im Minimum.

Die Hesse-Matrix muss numerisch bestimmt werden. Leider ist die Hesse-Matrix die `scipy.optimize.minimize` mit dem Ergebnis zusammen zurück gibt nur eine grobe Schätzung.

Wir nutzen `numdifftools.Hessian` um numerisch die Hesse-Matrix zu bestimmen.

`scipy.optimize.minimize` unterstützt mehrere Minimierungs-Algorithmen mit unterschiedlichen Fähigkeiten und Stärken. 
Für den Fall, dass man Grenzen für die Parameter (`bounds=`) angibt, ist der default der `L-BFGS-B` Algorithmus.

Dokumentation: 
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
* https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html

In [ ]:
from scipy.optimize import minimize

# a very small number to achive > 0 instead of >= 0
eps = np.finfo(np.float64).eps

result = minimize(
    neg_log_likelihood,
    args=(measured, ),
    x0=[130, 2, 30, 0.2], # here, the initial guess is required
    bounds=[
        (0, None),    # mean >= 0
        (eps, None),  # std > 0
        (eps, None),  # tau > 0
        (0, 1),       # 0 <= p <= 1
    ],
)

result

In [ ]:
from numdifftools import Hessian

hesse = Hessian(neg_log_likelihood)
cov = np.linalg.inv(hesse(result.x, measured))

In [ ]:
higgs_mass = result.x[0]
higgs_mass_unc = np.sqrt(cov[0, 0])
print(f'Higgs mass is {higgs_mass:.2f} ± {higgs_mass_unc:.2} GeV')

### Lösung mit iminuit


Iminuit ist ein Python Paket, dass Zugriff auf den Minimierungsalgorithmus "Minuit" aus dem ROOT Framework ermöglicht.

Hierzu ist keine Installation von ROOT notwendig. 

"Minuit" gilt – zumindest unter Teilchenphysiker:innen – als das "Non-Plus-Ultra" der Minimierer.

Siehe dazu auch: https://www.babushk.in/posts/adventures-physics-software-minuit.html 

`iminuit` stellt den Minimierer und Loss-Funktionen zur Verfügung, die es wesentlich einfacher machen,  
verschiedene Arten von Fits durchzuführen.

Außerdem berechnet es die Unsicherheiten über die Hesse-Matrix und stellt auch über das "minos" Interface Likelihood-Scans zur genaueren Bestimmung von Unsicherheiten über Likelihood-Ratio Tests zur Verfügung (dazu mehr in den nächsten Vorlesungen zum Thema Testen). 



Außerdem stellt es – gerade im Notebook – schöne Übersichten über den Fit-Vorgang zur Verfügung.

Für den Beginn einmal das gleiche Problem wie oben, nun gelöst mit `imuit`:

In [ ]:
from iminuit import Minuit
from iminuit.cost import UnbinnedNLL

# minuit's UnbinnedNLL takes directly the pdf and the observed data
loss = UnbinnedNLL(measured, pdf)

m = Minuit(loss, mean=130, std=2, tau=30, p=0.2)

# set bounds
m.limits['mean'] = (0, None)   # >= 0
m.limits['std'] = (eps, None)  # > 0
m.limits['tau'] = (eps, None)  # > 0
m.limits['p'] = (0, 1)

# perform minimization
m.migrad()

# perform likelihood scan for confidence intervals
m.minos()

In [ ]:
higgs_mass = m.values['mean']
higgs_mass_unc = m.errors['mean']

print(f'Higgs mass is {higgs_mass:.2f} ± {higgs_mass_unc:.2} GeV')

## Poisson-Likelihood-Fit einer histogrammierten Ereignisverteilung


Stehen die einzelnen Beobachtungen nicht zur Verfügung oder ist bei der Analyse von großen Datenmengen die Laufzeit relevant, kann auch ein sogenannter "gebinnter" Fit durchgeführt werden.

Hier wird die Wahrscheinlichts-Verteilung an das Histogramm angepasst.

Da ein Histogramm in jedem Bin ein Zählexperiment darstellt, erwarten wir eine Poisson-Statistik für jedes Bin.

Die kumulierte Wahrscheinlichkeitsdichte liefert zusammen mit der Gesamtereignis-Anzahl den Erwartungswert für jedes Histogramm-Bin, abhängig von den zu fittenden Parametern $\boldsymbol{\theta}$.

$$
\mathcal{L} = \prod_{i=1}^N \mathcal{P}(k=H_i, \lambda=\lambda_i(\boldsymbol{\theta}))
$$

mit 

$$
\lambda_i = N_\mathrm{total} \cdot (\mathrm{CDF}(b_i, \boldsymbol{\theta}) - \mathrm{CDF}(a_i, \boldsymbol{\theta}))
$$

Wobei $a_i$ und $b_i$ die Bin-Grenzen des $i$-ten Bins sind. Wir integrieren also die PDF in den einzelnen Bins und multiplizieren mit der Gesamtzahl der Ereignisse.


**Beispiel aus Lebensdauer kosmische Myonen (V01)**

Im Experiment wird direkt in Hardware ein Histogramm gemessen. 

Die einzelnen Lebensdauern stehen also nicht zur Verfügung.

Die Anleitung fordert einen Least-Squares-Fit and die Bin-Höhen. 
Dies liefert einen Erwartungstreuen Schätzer, so lange ein ungewichteter Fit durchgeführt wird.

In der Vergangenheit wurde empfohlen, einen gewichteten Fit unter der Annahme $\sigma_i = \sqrt{H_i}$ durchzuführen.

Dies ist falsch! Diese Methode liefert konsistent verzerrte Ergebnisse, da Unterfluktuationen stärker gewichtet werden als Überfluktuationen.

Die korrekte Methode ist der gebinnte Poisson-Likelihood Fit wie hier oder ein iteratives Least-Squares-Verfahren wie es in SMD-2 vorgestellt wird.

Vergleich der Methoden hier: https://gist.github.com/maxnoe/41730e6ca1fac01fc06f0feab5c3566d

In dem Versuch kommt es zusätzlich zu der erwarteten Exponential-Verteilung der Myon-Zerfälle noch zu einem gleichverteilten Untergrund durch koinzidente, nicht-zerfallende Myonen. 

In [ ]:
N = np.genfromtxt("resources/muon_data.txt")
t = np.arange(len(N) + 1) / 21.48

t = t[5:]
N = N[5:]

plt.figure()
plt.stairs(values=N, edges=t)
plt.xlabel('t / µs')
plt.ylabel('Ereignisanzahl')
None

Iminuit stellt für diesen Anwendungsfall die `BinnedNLL` Loss-Funktion zur Verfügung,
die genau die Poisson-Likelihood für ein gemessenes Histogram darstellt.

In [ ]:
from iminuit.cost import BinnedNLL
from scipy.stats import uniform

T_MIN, T_MAX = t[0], t[-1]

def cdf(x, tau, p):
    # normalize to 1 in histogram range
    cdf_min, cdf_max = expon.cdf([T_MIN, T_MAX], scale=tau) 
    norm = 1 / (cdf_max - cdf_min)
    
    # combine exponential signal with uniform background
    return (
        p * expon.cdf(x, scale=tau) * norm
        + (1 - p) * uniform.cdf(x, T_MIN, T_MAX)
    )


# histogram counds, histogram edges and cumulative distribution function
loss = BinnedNLL(N, t, cdf)

m = Minuit(loss, tau=2, p=0.99)
m.limits['tau'] = (eps, None)
m.limits['p'] = (0, 1)
m.migrad()
m.minos()

In [ ]:
muon_lifetime = m.values['tau']
muon_lifetime_unc = m.errors['tau']

print(f'Fit: τ = {muon_lifetime:.3f} ± {muon_lifetime_unc:.3f} µs')
print(f'Lit: τ = 2.1969811 ± 0.0000022 µs')

Likelihood-Scan für die Unsicherheiten:

In [ ]:
tau, ts, valid = m.mnprofile('tau', size=100)

In [ ]:
plt.figure()

plt.axvline(m.values['tau'], color='C1')
plt.plot(m.values['tau'], m.fval, color='C1', marker='o', zorder=3)

plt.axvline(m.values['tau'] + m.merrors['tau'].lower, color='C2')
plt.axvline(m.values['tau'] + m.merrors['tau'].upper, color='C2')
plt.axhline(m.fval + 1, color='C3')

plt.plot(tau, ts)

plt.xlabel('τ / µs')
plt.ylabel('NLL')
None

In [ ]:
plt.figure()
m.draw_mncontour('tau', 'p', size=250)